# Accessing the project directory on my Google Drive  

In [1]:
import os
from google.colab import drive

drive.mount('/drive', force_remount=False)
project_dir = "/drive/My Drive/RNNSeq2Seq/"

# change working directory to project_dir
os.chdir(project_dir)

Mounted at /drive


# Dependencies

In [2]:
from scripts.model import *
from scripts.dataloader import create_dataloader
from scripts.utils import *
from scripts.pytorch_utils import *
import matplotlib.pyplot as plt

# Helper function to help visualize 

In [3]:
def plot_log(log):
    len_range = len(log) -1 if "Best eval accu" in log else len(log)
    epoch_nums, train_loss, dev_loss = [], [], []
    train_acc, dev_acc = [], []

    for epoch in range(1, len_range+1):
        epoch_nums.append(epoch)
        train = log[f"Epoch#{epoch}"]["Train"]
        dev = log[f"Epoch#{epoch}"]["Eval"]
        train_loss.append(train['loss'])
        train_acc.append(train['abosulate accuracy'])

        dev_loss.append(dev['loss'])
        dev_acc.append(dev['abosulate accuracy'])

    ax1 = plt.subplot(211)
    ax1.plot(epoch_nums, train_loss, label="train loss")
    ax1.plot(epoch_nums, dev_loss, label="dev loss")
    ax1.legend()

    ax2 = plt.subplot(212)
    ax2.plot(epoch_nums, train_acc, label="train acc")
    ax2.plot(epoch_nums, dev_acc, label="dev acc")
    ax2.legend()
    plt.show()

In [4]:
folder = "tuning_data/"
experiment_num = 3
shuffle_batch = False

# must be the num of seq per seq length if shuffle_batch is False
# this "tr_batch_size" will be used for dev, and shuffled test sets
tr_batch_size = 250 
dev_batch_size = 500 if not shuffle_batch else tr_batch_size

In [11]:
# setups for training and evaluting

print_freq = 5
acc_threshold = 0.5 # for saving the best trained models 
max_epoch_num = 150 # max num of epoches 
train_acc_exit=0.92 # train acc exit threshold (eval_acc_exit must also be met)
eval_acc_exit=0.92 # dev acc exit threshold (train_acc_exit must also be met)
teacher_forcing_ratio = 1.0 # probability of using real symbol from target sequence for training

# RevStr

In [6]:
train = read_data(join(folder, "RevStr/train.txt"))
dev = read_data(join(folder, "RevStr/dev.txt"))
saved_model_fp = join(project_dir, "notebooks/Hyperparatemers Tuning/RevStr_model.pt")

train_dl = create_dataloader(train, batch_size=tr_batch_size, shuffle=shuffle_batch)
dev_dl = create_dataloader(dev, batch_size=dev_batch_size, shuffle=shuffle_batch)

## Hyperparameter set 1

In [7]:
in_vocab = ascii_lowercase
out_vocab = ascii_lowercase
in_vocab_size = len(in_vocab) + 2
out_vocab_size = len(out_vocab) + 2

hidden_size = 128
embd_dim = 128
num_layers = 1
rnn_type = "GRU"
dropout_rate = 0.0
bidirectional = False
use_attention = True
reduction_method = torch.sum

learning_rate = 1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = Encoder(in_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type, 
                  dropout_rate, bidirectional, 
                  reduction_method)
attention = Attention(hidden_size)
decoder = Decoder(out_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type,
                  attention, use_attention, 
                  dropout_rate)

model = Seq2Seq(encoder, decoder, device).to(device)
model.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
log = train_and_evaluate(model, train_dl, dev_dl, criterion, optimizer, 
                         saved_model_fp, acc_threshold, print_freq, max_epoch_num, 
                         train_acc_exit, eval_acc_exit, teacher_forcing_ratio)

Current epoch: 5, 
training performance: {'loss': 2.4457535564899446, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.019685683446004985, 'overlap rate': 0.25048580542206766}
evaluation performance: {'loss': 2.845012032985687, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.02015555864199996, 'overlap rate': 0.11595903150737286}

Current epoch: 10, 
training performance: {'loss': 1.5836026519536972, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.049587146891281006, 'overlap rate': 0.5089418184012174}
evaluation performance: {'loss': 3.4338837921619416, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.06301474669016897, 'overlap rate': 0.1685525428503752}

Current epoch: 15, 
training performance: {'loss': 1.2255381599068642, 'abosulate accuracy': 0.00020000000949949026, 'consecutive overlap rate': 0.09708716426976025, 'overlap rate': 0.6117292761802673}
evaluation performance: {'loss': 2.971750670671463, 'abosulate accuracy': 0.000900000031106174, 'c

KeyboardInterrupt: ignored

## Hyperparameter set 2


In [9]:
hidden_size = 256
embd_dim = 128
num_layers = 1
rnn_type = "GRU"
encoder = Encoder(in_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type, 
                  dropout_rate, bidirectional, 
                  reduction_method)
attention = Attention(hidden_size)
decoder = Decoder(out_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type,
                  attention, use_attention, 
                  dropout_rate)

model = Seq2Seq(encoder, decoder, device).to(device)
model.apply(init_weights)

learning_rate = 5e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
log = train_and_evaluate(model, train_dl, dev_dl, criterion, optimizer, 
                         saved_model_fp, acc_threshold, print_freq, max_epoch_num, 
                         train_acc_exit, eval_acc_exit, teacher_forcing_ratio)

Current epoch: 5, 
training performance: {'loss': 2.514810824394226, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.012963264109566807, 'overlap rate': 0.23202325757592918}
evaluation performance: {'loss': 2.7266683161258696, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.014252766885329038, 'overlap rate': 0.12868306115269662}

Current epoch: 10, 
training performance: {'loss': 1.8388170808553697, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.021562623570207506, 'overlap rate': 0.40079699754714965}
evaluation performance: {'loss': 2.663701021671295, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.02841340850573033, 'overlap rate': 0.17343423776328565}

Current epoch: 15, 
training performance: {'loss': 1.8207561999559403, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.036490919953212145, 'overlap rate': 0.4325233832001686}
evaluation performance: {'loss': 2.523798143863678, 'abosulate accuracy': 5.0000002374872565e-05, 'consecutive ove

KeyboardInterrupt: ignored

# RedStr



In [12]:
train = read_data(join(folder, "RedStr/train.txt"))
dev = read_data(join(folder, "RedStr/dev.txt"))
saved_model_fp = join(project_dir, "notebooks/Hyperparatemers Tuning/RevStr_model.pt")

train_dl = create_dataloader(train, batch_size=tr_batch_size, shuffle=shuffle_batch)
dev_dl = create_dataloader(dev, batch_size=dev_batch_size, shuffle=shuffle_batch)

## Hyperparameter set 1


In [13]:
hidden_size = 256
embd_dim = 128
num_layers = 1
rnn_type = "GRU"
encoder = Encoder(in_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type, 
                  dropout_rate, bidirectional, 
                  reduction_method)
attention = Attention(hidden_size)
decoder = Decoder(out_vocab_size, hidden_size, 
                  embd_dim, num_layers, rnn_type,
                  attention, use_attention, 
                  dropout_rate)

model = Seq2Seq(encoder, decoder, device).to(device)
model.apply(init_weights)

learning_rate = 5e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
log = train_and_evaluate(model, train_dl, dev_dl, criterion, optimizer, 
                         saved_model_fp, acc_threshold, print_freq, max_epoch_num, 
                         train_acc_exit, eval_acc_exit, teacher_forcing_ratio)

Current epoch: 5, 
training performance: {'loss': 2.3080928295850756, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.011379660642705857, 'overlap rate': 0.28606704957783224}
evaluation performance: {'loss': 2.4853764355182646, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.010527180065400899, 'overlap rate': 0.20534902438521385}

Current epoch: 10, 
training performance: {'loss': 1.4223325967788696, 'abosulate accuracy': 0.00010000000474974513, 'consecutive overlap rate': 0.06660987720824778, 'overlap rate': 0.5665697403252125}
evaluation performance: {'loss': 2.5068718910217287, 'abosulate accuracy': 0.0, 'consecutive overlap rate': 0.03691000221297145, 'overlap rate': 0.26349574252963065}

Current epoch: 15, 
training performance: {'loss': 0.3475031405687332, 'abosulate accuracy': 0.13110000389860943, 'consecutive overlap rate': 0.5724310437217355, 'overlap rate': 0.9151363596320152}
evaluation performance: {'loss': 1.7844072997570037, 'abosulate accuracy': 0.0498

KeyboardInterrupt: ignored